In [ ]:
!wget "https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz?"
!tar xvf "/content/genres.tar.gz?" -C "/content/"

In [2]:
!pip install shutup
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from librosa import feature
from glob import glob
import pandas as pd
import numpy as np
import librosa
import csv
import scipy

import shutup
import os
import shutil
shutup.please()

Feature Extraction from Audio data and saving them in their respective .csv 📁 file 🔉

In [4]:
genre = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

In [5]:
def GenerateFeatures (path_to_file):
  audio = librosa.util.example_audio_file() 
  y,sr=librosa.load(path_to_file)

  y_harmonic, y_percussive = librosa.effects.hpss(y)
  tempo, beat_frames = librosa.beat.beat_track(y=y_harmonic, sr=sr)

  chroma=librosa.feature.chroma_cens(y=y_harmonic, sr=sr)
  mfccs = librosa.feature.mfcc(y=y_harmonic, sr=sr, n_mfcc=13)
  cent = librosa.feature.spectral_centroid(y=y, sr=sr)
  contrast=librosa.feature.spectral_contrast(y=y_harmonic,sr=sr)
  rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
  zrate=librosa.feature.zero_crossing_rate(y_harmonic)

  chroma_mean=np.mean(chroma,axis=1)
  chroma_std=np.std(chroma,axis=1)
  chroma_df=pd.DataFrame()
  for i in range(0,12):
      chroma_df['chroma_mean_'+str(i)]=chroma_mean[i]
  for i in range(0,12):
      chroma_df['chroma_std_'+str(i)]=chroma_mean[i]
  chroma_df.loc[0]=np.concatenate((chroma_mean,chroma_std),axis=0)

  mfccs_mean=np.mean(mfccs,axis=1)
  mfccs_std=np.std(mfccs,axis=1)
  mfccs_df=pd.DataFrame()
  for i in range(0,13):
      mfccs_df['mfccs_mean_'+str(i)]=mfccs_mean[i]
  for i in range(0,13):
     mfccs_df['mfccs_std_'+str(i)]=mfccs_mean[i]
  mfccs_df.loc[0]=np.concatenate((mfccs_mean,mfccs_std),axis=0)

  cent_mean=np.mean(cent)
  cent_std=np.std(cent)
  cent_skew=scipy.stats.skew(cent,axis=1)[0]
  contrast_mean=np.mean(contrast,axis=1)
  contrast_std=np.std(contrast,axis=1)  

  rolloff_mean=np.mean(rolloff)
  rolloff_std=np.std(rolloff)
  rolloff_skew=scipy.stats.skew(rolloff,axis=1)[0]

  spectral_df=pd.DataFrame()
  collist=['cent_mean','cent_std','cent_skew']
  for i in range(0,7):
      collist.append('contrast_mean_'+str(i))
  for i in range(0,7):
      collist.append('contrast_std_'+str(i))
  collist=collist+['rolloff_mean','rolloff_std','rolloff_skew']
  for c in collist:
      spectral_df[c]=0
  data=np.concatenate(([cent_mean,cent_std,cent_skew],contrast_mean,contrast_std,[rolloff_mean,rolloff_std,rolloff_std]),axis=0)
  spectral_df.loc[0]=data

  zrate_mean=np.mean(zrate)
  zrate_std=np.std(zrate)
  zrate_skew=scipy.stats.skew(zrate,axis=1)[0]

  zrate_df=pd.DataFrame()
  zrate_df['zrate_mean']=0
  zrate_df['zrate_std']=0
  zrate_df['zrate_skew']=0
  zrate_df.loc[0]=[zrate_mean,zrate_std,zrate_skew]

  beat_df=pd.DataFrame()
  beat_df['tempo']=tempo
  beat_df.loc[0]=tempo

  final_df=pd.concat((chroma_df,mfccs_df,spectral_df,zrate_df,beat_df),axis=1)

  return final_df

In [6]:
GenerateFeatures('/content/genres/blues/blues.00000.au').columns

Index(['chroma_mean_0', 'chroma_mean_1', 'chroma_mean_2', 'chroma_mean_3',
       'chroma_mean_4', 'chroma_mean_5', 'chroma_mean_6', 'chroma_mean_7',
       'chroma_mean_8', 'chroma_mean_9', 'chroma_mean_10', 'chroma_mean_11',
       'chroma_std_0', 'chroma_std_1', 'chroma_std_2', 'chroma_std_3',
       'chroma_std_4', 'chroma_std_5', 'chroma_std_6', 'chroma_std_7',
       'chroma_std_8', 'chroma_std_9', 'chroma_std_10', 'chroma_std_11',
       'mfccs_mean_0', 'mfccs_mean_1', 'mfccs_mean_2', 'mfccs_mean_3',
       'mfccs_mean_4', 'mfccs_mean_5', 'mfccs_mean_6', 'mfccs_mean_7',
       'mfccs_mean_8', 'mfccs_mean_9', 'mfccs_mean_10', 'mfccs_mean_11',
       'mfccs_mean_12', 'mfccs_std_0', 'mfccs_std_1', 'mfccs_std_2',
       'mfccs_std_3', 'mfccs_std_4', 'mfccs_std_5', 'mfccs_std_6',
       'mfccs_std_7', 'mfccs_std_8', 'mfccs_std_9', 'mfccs_std_10',
       'mfccs_std_11', 'mfccs_std_12', 'cent_mean', 'cent_std', 'cent_skew',
       'contrast_mean_0', 'contrast_mean_1', 'contrast_mean_2'

Data Genrator

In [7]:
dir = '/content/genres/'
columns = ['chroma_mean_0', 'chroma_mean_1', 'chroma_mean_2', 'chroma_mean_3',
       'chroma_mean_4', 'chroma_mean_5', 'chroma_mean_6', 'chroma_mean_7',
       'chroma_mean_8', 'chroma_mean_9', 'chroma_mean_10', 'chroma_mean_11',
       'chroma_std_0', 'chroma_std_1', 'chroma_std_2', 'chroma_std_3',
       'chroma_std_4', 'chroma_std_5', 'chroma_std_6', 'chroma_std_7',
       'chroma_std_8', 'chroma_std_9', 'chroma_std_10', 'chroma_std_11',
       'mfccs_mean_0', 'mfccs_mean_1', 'mfccs_mean_2', 'mfccs_mean_3',
       'mfccs_mean_4', 'mfccs_mean_5', 'mfccs_mean_6', 'mfccs_mean_7',
       'mfccs_mean_8', 'mfccs_mean_9', 'mfccs_mean_10', 'mfccs_mean_11',
       'mfccs_mean_12', 'mfccs_std_0', 'mfccs_std_1', 'mfccs_std_2',
       'mfccs_std_3', 'mfccs_std_4', 'mfccs_std_5', 'mfccs_std_6',
       'mfccs_std_7', 'mfccs_std_8', 'mfccs_std_9', 'mfccs_std_10',
       'mfccs_std_11', 'mfccs_std_12', 'cent_mean', 'cent_std', 'cent_skew',
       'contrast_mean_0', 'contrast_mean_1', 'contrast_mean_2',
       'contrast_mean_3', 'contrast_mean_4', 'contrast_mean_5',
       'contrast_mean_6', 'contrast_std_0', 'contrast_std_1', 'contrast_std_2',
       'contrast_std_3', 'contrast_std_4', 'contrast_std_5', 'contrast_std_6',
       'rolloff_mean', 'rolloff_std', 'rolloff_skew', 'zrate_mean',
       'zrate_std', 'zrate_skew', 'tempo']

for gen in genre:
  print (f"Current gen: {gen}")
  path = dir + gen

  files = os.listdir(path+'/')
  GenreFrame = pd.DataFrame(columns=columns)

  for clip in files:
    clip_path = dir + gen + '/' + clip

    print(f'Extracting features from {clip_path}')
    clipDataFrame = GenerateFeatures(clip_path)
    clipDataFrame['label'] = gen
  
    GenreFrame = pd.concat((GenreFrame, clipDataFrame), axis=0)

  GenreFrame.to_csv('/content/' + gen + '.csv')
  print(f'Label: {gen} saved at /content/{gen}.csv\n\n\n\n')
  shutil.copy (f"/content/{gen}.csv", f"/content/drive/MyDrive/IITK DL datasets/{gen}.csv")


Reading .csv file and adding a new column type ✔

In [43]:
blues = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/blues.csv")
classical = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/classical.csv")
country = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/country.csv")
disco = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/disco.csv")
hiphop = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/hiphop.csv")
jazz = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/jazz.csv")
metal = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/metal.csv")
pop = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/pop.csv")
reggae = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/reggae.csv")
rock = pd.read_csv("/content/drive/MyDrive/IITK DL datasets/rock.csv")

blues.drop(["Unnamed: 0"], inplace=True, axis=1)
classical.drop(["Unnamed: 0"], inplace=True, axis=1)
country.drop(["Unnamed: 0"], inplace=True, axis=1)
disco.drop(["Unnamed: 0"], inplace=True, axis=1)
hiphop.drop(["Unnamed: 0"], inplace=True, axis=1)
jazz.drop(["Unnamed: 0"], inplace=True, axis=1)
metal.drop(["Unnamed: 0"], inplace=True, axis=1)
pop.drop(["Unnamed: 0"], inplace=True, axis=1)
reggae.drop(["Unnamed: 0"], inplace=True, axis=1)
rock.drop(["Unnamed: 0"], inplace=True, axis=1)

blues['label'] = 'blues'
classical['label'] = 'classical'
country['label'] = 'country'
disco['label'] = 'disco'
hiphop['label'] = 'hiphop'
jazz['label'] = 'jazz'
metal['label'] = 'metal'
pop['label'] = 'pop'
reggae['label'] = 'reggae'
rock['label'] = 'rock'

Creating The Main Dataframe for training our Model 🚩

In [44]:
genres = [blues, 
          classical, 
          country,
          hiphop, 
          jazz,
          metal, 
          pop,
          reggae,
          disco,
          rock]

final_df = pd.concat(genres)

In [45]:
final_df.columns

Index(['chroma_mean_0', 'chroma_mean_1', 'chroma_mean_2', 'chroma_mean_3',
       'chroma_mean_4', 'chroma_mean_5', 'chroma_mean_6', 'chroma_mean_7',
       'chroma_mean_8', 'chroma_mean_9', 'chroma_mean_10', 'chroma_mean_11',
       'chroma_std_0', 'chroma_std_1', 'chroma_std_2', 'chroma_std_3',
       'chroma_std_4', 'chroma_std_5', 'chroma_std_6', 'chroma_std_7',
       'chroma_std_8', 'chroma_std_9', 'chroma_std_10', 'chroma_std_11',
       'mfccs_mean_0', 'mfccs_mean_1', 'mfccs_mean_2', 'mfccs_mean_3',
       'mfccs_mean_4', 'mfccs_mean_5', 'mfccs_mean_6', 'mfccs_mean_7',
       'mfccs_mean_8', 'mfccs_mean_9', 'mfccs_mean_10', 'mfccs_mean_11',
       'mfccs_mean_12', 'mfccs_std_0', 'mfccs_std_1', 'mfccs_std_2',
       'mfccs_std_3', 'mfccs_std_4', 'mfccs_std_5', 'mfccs_std_6',
       'mfccs_std_7', 'mfccs_std_8', 'mfccs_std_9', 'mfccs_std_10',
       'mfccs_std_11', 'mfccs_std_12', 'cent_mean', 'cent_std', 'cent_skew',
       'contrast_mean_0', 'contrast_mean_1', 'contrast_mean_2'

Encoding labels of Target column ✈

In [46]:
final_df.sample(20)

,chroma_mean_0,chroma_mean_1,chroma_mean_2,chroma_mean_3,chroma_mean_4,chroma_mean_5,chroma_mean_6,chroma_mean_7,chroma_mean_8,chroma_mean_9,...,contrast_std_5,contrast_std_6,rolloff_mean,rolloff_std,rolloff_skew,zrate_mean,zrate_std,zrate_skew,tempo,label
84,0.146691,0.190941,0.253665,0.242875,0.162607,0.148053,0.139288,0.144326,0.260059,0.336050,...,3.353145,3.580907,2443.177544,902.406450,902.406450,0.032688,0.015927,1.100622,95.703125,jazz
42,0.240252,0.136283,0.197336,0.139170,0.303921,0.143400,0.228281,0.364495,0.277468,0.273754,...,2.859025,2.633459,5291.114358,991.339070,991.339070,0.117712,0.036162,2.228824,103.359375,metal
16,0.197543,0.255582,0.269635,0.195362,0.384643,0.248956,0.167442,0.232984,0.275626,0.320376,...,3.424264,3.231101,5113.935898,1227.565295,1227.565295,0.105971,0.042322,0.642499,135.999178,disco
82,0.047321,0.073755,0.094531,0.450756,0.293272,0.079767,0.039265,0.160462,0.523101,0.269207,...,2.453349,5.058617,4447.838817,1083.182591,1083.182591,0.059825,0.029386,0.291689,117.453835,metal
42,0.232848,0.402612,0.355055,0.260005,0.270997,0.135102,0.110403,0.185295,0.337231,0.257221,...,2.792755,3.494094,4068.834458,2163.905251,2163.905251,0.046215,0.055914,3.438228,75.999540,reggae
98,0.246911,0.252057,0.434070,0.172102,0.163476,0.170781,0.089668,0.207330,0.103658,0.224374,...,4.393336,4.579843,1809.963147,749.228207,749.228207,0.043154,0.022626,1.013635,123.046875,jazz
74,0.305916,0.313873,0.180063,0.207780,0.302883,0.091915,0.223900,0.157998,0.139015,0.216623,...,3.246650,2.821427,2754.859418,1469.929754,1469.929754,0.039456,0.019448,1.112195,112.347147,blues
80,0.165598,0.344555,0.140105,0.202332,0.321263,0.140429,0.229249,0.152634,0.307830,0.337455,...,4.958507,1.753282,6442.857612,1798.863876,1798.863876,0.108711,0.054124,1.770280,92.285156,pop
39,0.130898,0.209171,0.261850,0.352120,0.305581,0.328245,0.342078,0.251632,0.263725,0.249348,...,3.009097,2.601774,5384.782958,874.471755,874.471755,0.131161,0.034253,0.547397,112.347147,metal
58,0.191257,0.096700,0.270225,0.139363,0.209267,0.223354,0.212258,0.316568,0.172161,0.219651,...,4.487261,4.030969,2489.766203,658.841171,658.841171,0.086315,0.031371,0.624755,112.347147,classical


Encoding Labels of Target variable ⚓

In [49]:
convertor = LabelEncoder()
class_list = final_df.iloc[:,-1]
y = convertor.fit_transform(class_list)

Fitting Rest of the columns in StanderdScaler (Using fit_transform for that)

In [50]:
fit = StandardScaler()
X = fit.fit_transform(np.array(final_df.iloc[:,:-1], dtype = float))

In [51]:
X

array([[ 0.82833167, -1.33495166, -1.28353263, ..., -1.47272182,
         0.40958727, -0.51955897],
       [ 0.76552063, -1.29965731, -0.58771025, ...,  0.05443765,
        -0.66389224,  2.18518953],
       [ 0.13282168, -0.8275167 , -0.73332123, ..., -0.24657498,
         1.09981381, -1.91463978],
       ...,
       [ 0.71974078, -0.6994479 , -0.80776934, ..., -0.42292   ,
        -0.77951094,  0.02139073],
       [-1.90772386,  1.30226094, -1.81263156, ...,  0.18421847,
         0.79699784, -0.35492211],
       [-0.2659568 , -1.40462779,  0.03128313, ..., -0.12075206,
         0.27557264, -1.63327894]])

Splitting Our Dataset for Training (x_train & y_train) and Testing (y_train & y_test)

In [52]:
x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y ,test_size=0.1)

Creating TrainModel Function for training our RNN model.

In [53]:
def trainModel (model, epochs, optimizer, verbose):
  batch_size = 128

  model.compile(optimizer,
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')

  return model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 epochs=epochs, batch_size=batch_size, verbose=verbose)

Our Sequential Model ⚖

In [54]:
model = keras.models.Sequential([
                             keras.layers.Dense(512, activation='relu', input_shape=(x_train.shape[1],)),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(256, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(128, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Dense(64, activation='relu'),
                             keras.layers.Dropout(0.2),

                             keras.layers.Flatten(),
                             keras.layers.Dense(10, activation='softmax')
                          ])
print(model.summary())
model_history = trainModel(model=model, epochs=150, optimizer='adam', verbose=2)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 512)               38400     
                                                                 
 dropout_20 (Dropout)        (None, 512)               0         
                                                                 
 dense_26 (Dense)            (None, 256)               131328    
                                                                 
 dropout_21 (Dropout)        (None, 256)               0         
                                                                 
 dense_27 (Dense)            (None, 128)               32896     
                                                                 
 dropout_22 (Dropout)        (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 64)               

Summary of Model ✉

In [55]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 512)               38400     
                                                                 
 dropout_20 (Dropout)        (None, 512)               0         
                                                                 
 dense_26 (Dense)            (None, 256)               131328    
                                                                 
 dropout_21 (Dropout)        (None, 256)               0         
                                                                 
 dense_27 (Dense)            (None, 128)               32896     
                                                                 
 dropout_22 (Dropout)        (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 64)               

Model evaluation (Increase test size to get better evaluation ✅

In [56]:
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=128)
print (f"\nThe test loss is : {round(test_loss, 2)}")
print(f"The best test accuracy is : {round(test_acc*100, 2)}%")

1/1 [==============================] - 0s 17ms/step - loss: 1.2796 - accuracy: 0.7900

The test loss is : 1.28
The best test accuracy is : 79.0%


Preparing data for pridiction ☕

In [57]:
def makeAudio_to_Data (path):

  newDataframe = GenerateFeatures (path)
  nextDataframe = GenerateFeatures ('/content/genres/disco/disco.00000.au')

  numpyArray = fit.fit_transform(np.array(pd.concat([newDataframe, nextDataframe], axis=0), dtype=float))

  return numpyArray, newDataframe

Calling makeAudio_to_Data function

In [58]:
frame, data = makeAudio_to_Data ("/content/genres/jazz/jazz.00001.au")

Predicting

In [59]:
arr = model.predict(frame)
lis = arr.tolist()
print(genre[lis[0].index(max(lis[0]))].capitalize())

Jazz


In [60]:
data.sample()

,chroma_mean_0,chroma_mean_1,chroma_mean_2,chroma_mean_3,chroma_mean_4,chroma_mean_5,chroma_mean_6,chroma_mean_7,chroma_mean_8,chroma_mean_9,...,contrast_std_4,contrast_std_5,contrast_std_6,rolloff_mean,rolloff_std,rolloff_skew,zrate_mean,zrate_std,zrate_skew,tempo
0,0.298391,0.239644,0.282745,0.200693,0.136922,0.355245,0.323364,0.24725,0.165014,0.228327,...,6.105826,5.602013,4.601369,2410.827778,1333.896432,1333.896432,0.055701,0.030833,0.748121,135.999178
